In [1]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import xlrd 

%matplotlib inline
os.getcwd()

'C:\\Users\\Charlie\\Documents\\42577BusAna\\Project'

In [2]:
names=['Station Name', 'MAC Address', 'Org Name', 'Start Date', 'Start Time Zone', 
       'End Date', 'End Time Zone', 'Transaction Date', 'Total Duration', 'Charging Time', 
       'Energy (kWh)', 'GHG Savings (kg)', 'Gasoline Savings (gallons)', 
       'Port Type', 'Port Number', 'Plug Type', 'EVSE ID', 'Address 1', 'City', 'State/Province', 'Postal Code', 
       'Country', 'Latitude', 'Longitude', 'Currency', 'Fee', 'Ended By', 'Plug In Event Id', 'Driver Postal Code', 
       'User ID', 'County', 'System S/N', 'Model Number']
f=pd.read_csv("EVChargingStationUsage.csv", header=0, names=names)

print(f.dtypes)
print(f.columns.tolist())
f.head()

Station Name                   object
MAC Address                    object
Org Name                       object
Start Date                     object
Start Time Zone                object
End Date                       object
End Time Zone                  object
Transaction Date               object
Total Duration                 object
Charging Time                  object
Energy (kWh)                  float64
GHG Savings (kg)              float64
Gasoline Savings (gallons)    float64
Port Type                      object
Port Number                     int64
Plug Type                      object
EVSE ID                       float64
Address 1                      object
City                           object
State/Province                 object
Postal Code                     int64
Country                        object
Latitude                      float64
Longitude                     float64
Currency                       object
Fee                           float64
Ended By    

C:\Users\Charlie\AppData\Local\Temp\ipykernel_26272\2501802635.py:7: DtypeWarning: Columns (29,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  f=pd.read_csv("EVChargingStationUsage.csv", header=0, names=names)


,Station Name,MAC Address,Org Name,Start Date,Start Time Zone,End Date,End Time Zone,Transaction Date,Total Duration,Charging Time,...,Longitude,Currency,Fee,Ended By,Plug In Event Id,Driver Postal Code,User ID,County,System S/N,Model Number
0,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/29/2011 20:17,PDT,7/29/2011 23:20,PDT,7/29/2011 23:20,3:03:32,1:54:03,...,-122.160309,USD,0.0,Plug Out at Vehicle,3,95124.0,3284.0,NaN,NaN,NaN
1,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 0:00,PDT,7/30/2011 0:02,PDT,7/30/2011 0:02,0:02:06,0:01:54,...,-122.160309,USD,0.0,Customer,4,94301.0,4169.0,NaN,NaN,NaN
2,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 8:16,PDT,7/30/2011 12:34,PDT,7/30/2011 12:34,4:17:32,4:17:28,...,-122.160309,USD,0.0,Plug Out at Vehicle,5,94301.0,4169.0,NaN,NaN,NaN
3,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 14:51,PDT,7/30/2011 16:55,PDT,7/30/2011 16:55,2:03:24,2:02:58,...,-122.160309,USD,0.0,Customer,6,94302.0,2545.0,NaN,NaN,NaN
4,PALO ALTO CA / HAMILTON #1,000D:6F00:015A:9D76,City of Palo Alto,7/30/2011 18:51,PDT,7/30/2011 20:03,PDT,7/30/2011 20:03,1:11:24,0:43:54,...,-122.160309,USD,0.0,Plug Out at Vehicle,7,94043.0,3765.0,NaN,NaN,NaN


In [3]:
def process_station_name(station_name):
    # Remove all spaces
    station_name = station_name.replace(" ", "")

    # Splits the data at the / and only retreives what comes after and removes all spaces
    if '/' in station_name:
        # Split at the '/' and return the part after it if it exists
        parts = station_name.split("/")
        if len(parts) > 1:  # Check if there is a second part
            return parts[1]
    # If no '/' or only one part, return the original name
    return station_name


def process_start_date(df):
    #Converts 'Start Date' column into datetime format, and creates new timeframe & month & year value for later processing
    df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')

    # create columns in the DataFrame >> presents data differently for future use
    new_date_columns = pd.DataFrame({
        'StartTimestamp': df['Start Date'].astype(int) // 10**9,
        'SMonth': df['Start Date'].dt.month,
        'SYear': df['Start Date'].dt.year,
    })
  
    # DUPLICATION STOPPER!! Otherwise datasets is ever growing everytime this is run
    date_columns = new_date_columns.keys()
    df.drop(columns=[col for col in date_columns if col in df.columns], inplace=True)

    # Determine the index of Start date to insert new columns right behind
    start_date_index = df.columns.get_loc('Start Date') + 1
    for column, value in new_date_columns.items():
        df.insert(start_date_index, column, value)
        start_date_index += 1  # Increment index for next insertion

    return df


def convert_decimal_dates(value):
    # both the decimal and the normal date are str originally 
    # this identifies the "hidden" float aka serial number so that neither one nor the other gets converted to NaT during to_datetime 
    if isinstance(value, str):
        try:
            # Attempt to convert to float to check if it's a serial date
            float_value = float(value)
            # If conversion is successful, treat it as a serial date
            return pd.to_datetime('1899-12-30') + pd.to_timedelta(float_value, unit='D')
        except ValueError:
            # If conversion fails, return the original string
            return value
    elif isinstance(value, (float, int)):
        # If it's already a float or int, convert it to datetime
        return pd.to_datetime('1899-12-30') + pd.to_timedelta(value, unit='D')
    else:
        # Handle unexpected types by returning the original value
        return value


def process_end_date(df):
    # Converts 'End Date' column into datetime format, and creates new timeframe & month & year value for later processing

    # Conversion of serial values into datetime format
    f['End Date'] = f['End Date'].apply(convert_decimal_dates)
    f['End Date'] = pd.to_datetime(f['End Date'])

    # Extract day, month, year, and time as new DataFrame columns
    new_date_columns = pd.DataFrame({
        'EndTimestamp': df['End Date'].astype(int) // 10**9,
        'EMonth': df['End Date'].dt.month,
        'EYear': df['End Date'].dt.year,
    })

    date_columns = new_date_columns.keys()
    df.drop(columns=[col for col in date_columns if col in df.columns], inplace=True)

    # Determine the index to insert new columns
    start_date_index = df.columns.get_loc('End Date') + 1

    # Insert the new date columns into the DataFrame
    for column, value in new_date_columns.items():
        df.insert(start_date_index, column, value)
        start_date_index += 1  # Increment index for next insertion

    return df


In [4]:
print(f)

                          Station Name          MAC Address  \
0           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76   
1           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76   
2           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76   
3           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76   
4           PALO ALTO CA / HAMILTON #1  000D:6F00:015A:9D76   
...                                ...                  ...   
259410  PALO ALTO CA / TED THOMPSON #3  0024:B100:0002:9F81   
259411       PALO ALTO CA / WEBSTER #1  0024:B100:0002:4233   
259412           PALO ALTO CA / MPL #4  0024:B100:0003:CD37   
259413       PALO ALTO CA / WEBSTER #1  0024:B100:0002:4233   
259414     PALO ALTO CA / CAMBRIDGE #4  0024:B100:0003:3A0A   

                  Org Name        Start Date Start Time Zone  \
0        City of Palo Alto   7/29/2011 20:17             PDT   
1        City of Palo Alto    7/30/2011 0:00             PDT   
2        City of Palo Alto    7/30/2011 8:16       

In [5]:
# Apply the function to the 'Station Name' column
f['Station Name'] = f['Station Name'].apply(process_station_name)
print("Cleaning Station Name column is systematised")
f = process_start_date(f)
print("Start date converted to datetime64[ns] format and new columns generated")
f = process_end_date(f)
print("End date converted to datetime64[ns], with serial values accounted for, format and new columns generated")


Cleaning Station Name column is systematised
Start date converted to datetime64[ns] format and new columns generated
End date converted to datetime64[ns], with serial values accounted for, format and new columns generated


In [6]:
print(f)

         Station Name          MAC Address            Org Name  \
0          HAMILTON#1  000D:6F00:015A:9D76   City of Palo Alto   
1          HAMILTON#1  000D:6F00:015A:9D76   City of Palo Alto   
2          HAMILTON#1  000D:6F00:015A:9D76   City of Palo Alto   
3          HAMILTON#1  000D:6F00:015A:9D76   City of Palo Alto   
4          HAMILTON#1  000D:6F00:015A:9D76   City of Palo Alto   
...               ...                  ...                 ...   
259410  TEDTHOMPSON#3  0024:B100:0002:9F81  City of Palo Alto    
259411      WEBSTER#1  0024:B100:0002:4233  City of Palo Alto    
259412          MPL#4  0024:B100:0003:CD37  City of Palo Alto    
259413      WEBSTER#1  0024:B100:0002:4233  City of Palo Alto    
259414    CAMBRIDGE#4  0024:B100:0003:3A0A  City of Palo Alto    

                Start Date  StartTimestamp  SMonth  SYear Start Time Zone  \
0      2011-07-29 20:17:00      1311970620       7   2011             PDT   
1      2011-07-30 00:00:00      1311984000       7   

In [8]:
## Replaces the serial numbers with date and makes the right columns etc

def process_transaction_datev1(df):
    """
    Converts 'End Date' column in the DataFrame to datetime format, handling serial numbers,
    extracts day, month, year, and time, and replaces the original 'End Date'
    column with the new columns.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the 'End Date' column.

    Returns:
    pd.DataFrame: The modified DataFrame with new date columns.
    """

    # Conversion of serial values into datetime format
    f['Transaction Date'] = f['Transaction Date'].apply(convert_decimal_dates)
    f['Transaction Date'] = pd.to_datetime(f['Transaction Date'])

    # Apply conversion to each value in 'End Date' column

    # Extract day, month, year, and time as new DataFrame columns
    new_date_columns = pd.DataFrame({
        'TransactionTimestamp': df['Transaction Date'].astype(int) // 10**9,
        'TMonth': df['Transaction Date'].dt.month,
        'TYear': df['Transaction Date'].dt.year,
    })

    date_columns = new_date_columns.keys()
    df.drop(columns=[col for col in date_columns if col in df.columns], inplace=True)

    # Determine the index to insert new columns
    start_date_index = df.columns.get_loc('Transaction Date') + 1

    # Insert the new date columns into the DataFrame
    for column, value in new_date_columns.items():
        df.insert(start_date_index, column, value)
        start_date_index += 1  # Increment index for next insertion

    return df

# Apply the function
f = process_transaction_datev1(f)


print(f['Transaction Date'])
print(f.loc[55276,'Transaction Date'])
print(f.loc[55277,'Transaction Date'])

0        2011-07-29 23:20:00
1        2011-07-30 00:02:00
2        2011-07-30 12:34:00
3        2011-07-30 16:55:00
4        2011-07-30 20:03:00
                 ...        
259410   2020-12-31 17:16:00
259411   2020-12-31 20:32:00
259412   2021-01-01 10:57:00
259413   2020-12-31 21:56:00
259414   2020-12-31 19:02:00
Name: Transaction Date, Length: 259415, dtype: datetime64[ns]
NaT
2016-01-25 18:50:00


In [ ]:
## Replaces the serial numbers with date and makes the right columns etc
## ALSO COPY THE DATA FROM END DATE INTO THE ROWS WITH NaT

def process_transaction_datev2(df):
    """
    Converts 'End Date' column in the DataFrame to datetime format, handling serial numbers,
    extracts day, month, year, and time, and replaces the original 'End Date'
    column with the new columns.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the 'End Date' column.

    Returns:
    pd.DataFrame: The modified DataFrame with new date columns.
    """

    # Conversion of serial values into datetime format
    f['Transaction Date'] = f['Transaction Date'].apply(convert_decimal_dates)
    f['Transaction Date'] = pd.to_datetime(f['Transaction Date'])
    f['Transaction Date'] = f['Transaction Date'].fillna(pd.to_datetime(f['End Date']))

    # Apply conversion to each value in 'End Date' column

    # Extract day, month, year, and time as new DataFrame columns
    new_date_columns = pd.DataFrame({
        'TransactionTimestamp': df['Transaction Date'].astype(int) // 10**9,
        'TMonth': df['Transaction Date'].dt.month,
        'TYear': df['Transaction Date'].dt.year,
    })

    date_columns = new_date_columns.keys()
    df.drop(columns=[col for col in date_columns if col in df.columns], inplace=True)

    # Determine the index to insert new columns
    start_date_index = df.columns.get_loc('Transaction Date') + 1

    # Insert the new date columns into the DataFrame
    for column, value in new_date_columns.items():
        df.insert(start_date_index, column, value)
        start_date_index += 1  # Increment index for next insertion

    return df

# Apply the function
f = process_transaction_datev2(f)


print(f['Transaction Date'])
print(f.loc[55276,'Transaction Date'])
print(f.loc[55277,'Transaction Date'])

In [ ]:
print(f.dtypes)


In [9]:
f.to_csv("EVChargingStationUsagePrep.csv", mode='w')